In [1]:
import numpy as np
from keras.utils import np_utils
import keras.preprocessing.sequence as sq

Using TensorFlow backend.


In [2]:
npzdata=np.load("data.npz")

In [3]:
data=npzdata['data']
labels=npzdata['labels']

In [4]:
maxlen=150
data=sq.pad_sequences(data,maxlen=maxlen,padding='post',truncating='post',dtype='float64')
data=data.reshape(data.shape[0],data.shape[1],1)
dict_label={}
n=0
set_labels=list(set(labels))
for l in set_labels:
    dict_label[l]=n
    n+=1

In [5]:
labels_new=[]
for l in labels:
    labels_new.append(dict_label[l])
label_1=np_utils.to_categorical(labels_new)

In [6]:
def generateData(data,labels,batch_size=32):
    nb_instances = data.shape[0]
    nb_classes = labels.shape[1]
    sample_shape = data[0].shape
    batch_data_shape = tuple([batch_size] + list(sample_shape))
    batch_label_shape = (batch_size, nb_classes)
    print(batch_data_shape)
    print(batch_label_shape)
    # Infinite loop
    while True:
        # Generate an exploration order
        indices = np.arange(nb_instances)
        np.random.shuffle(indices)

        # Generate batches
        imax = int(len(indices) / batch_size)
        for i in range(imax):
            # Form a batch
            x = np.empty(batch_data_shape)
            y = np.empty(batch_label_shape)
            for j, k in enumerate(indices[i * batch_size: (i + 1) * batch_size]):
                x[j] = data[k]
                y[j] = labels[k]
            if x.shape != batch_data_shape:
                print(x.shape)
                exit(0)
            yield x, y

In [7]:
from sklearn import model_selection
X_train, x_test, Y_train, y_test = model_selection.train_test_split(data, label_1, test_size=0.15)
x_train, x_val, y_train, y_val = model_selection.train_test_split(X_train, Y_train, test_size=0.18)

trainGen=generateData(x_train,y_train)
valGen=generateData(x_val,y_val)
testGen=generateData(x_test,y_test)

In [8]:
from keras.layers.core import Dense, Dropout,Permute,Lambda,RepeatVector,Flatten
from keras.layers.recurrent import LSTM
from keras.layers import Input,multiply
from keras.models import *
from keras.optimizers import RMSprop
import keras.backend as K

In [9]:
input_length=maxlen
input_dim=1

In [10]:
inputs = Input(shape=(input_length, input_dim))

In [11]:
lstm_activitions=LSTM(units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True,dropout=0.2)(inputs)

In [12]:
attention= Permute((2, 1))(lstm_activitions)

In [13]:
attention = Dense(input_length, activation='softmax')(attention)

In [14]:
attention = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(attention)
attention = RepeatVector(input_dim)(attention)

In [15]:
a_probs = Permute((2, 1), name='attention_vec')(attention)


In [16]:
output_attention_mul  = multiply([lstm_activitions, a_probs])

In [17]:
nb_classes=label_1.shape[1]

In [18]:
nb_classes

101

In [19]:
lstm_activations_2 = LSTM(units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True,dropout=0.2)(output_attention_mul)

In [26]:
attention_2= Permute((2, 1))(lstm_activations_2)
attention_2 = Dense(input_length, activation='softmax')(attention_2)
attention_2 = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction_2')(attention_2)
attention_2 = RepeatVector(input_dim)(attention_2)
a_probs_2 = Permute((2, 1), name='attention_vec_2')(attention_2)
output_attention_mul  = multiply([lstm_activations_2, a_probs_2])
ouputs=Flatten()(output_attention_mul)

In [27]:
y=Dense(nb_classes,activation='softmax',name='output')(ouputs)

In [28]:
model=Model(inputs=[inputs],outputs=[y])

In [29]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 1)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 150, 128)     66560       input_1[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 128, 150)     0           lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128, 150)     22650       permute_1[0][0]                  
__________________________________________________________________________________________________
dim_reduct

In [30]:
metrics=['accuracy']

In [31]:
optimizer=RMSprop(lr=0.001,decay=0.0)

In [32]:
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=metrics)

In [33]:
print (model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 1)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 150, 128)     66560       input_1[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 128, 150)     0           lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128, 150)     22650       permute_1[0][0]                  
__________________________________________________________________________________________________
dim_reduct

In [ ]:
history=model.fit_generator(generator=trainGen,steps_per_epoch=x_train.shape[0]//32,validation_data=valGen,validation_steps=x_val.shape[0]//32,epochs=5)

Epoch 1/5
(32, 150, 1)
(32, 101)
(32, 150, 1)
(32, 101)
1196/5445 [=====>........................] - ETA: 36:45 - loss: 4.2513 - acc: 0.0308